# Local Trainng first

In [331]:
# Loading datasets

from ImageDataset import ImageDataset
from TextDataset import TokenizerDataset

# flicke image
img_ds = ImageDataset(center_crop=True, center_crop_shape=(224, 224), return_label=True) # return label = True
img_ds.load('flicker_data/flicker_toy_data/flicker/images/')
# text
txt_ds = TokenizerDataset(return_label=False) 
txt_ds.load('flicker_data/flicker_toy_data/text.csv')

In [332]:
print(len(img_ds))
print(img_ds[0])
print(img_ds.get_classes())
print(img_ds.get_sample_ids()[0: 10])

215
(tensor([[[0.5059, 0.5176, 0.5137,  ..., 0.4941, 0.5020, 0.5059],
         [0.4980, 0.5020, 0.4980,  ..., 0.4824, 0.5020, 0.5059],
         [0.5059, 0.4863, 0.4902,  ..., 0.4980, 0.4980, 0.5137],
         ...,
         [0.7843, 0.7922, 0.7529,  ..., 0.1412, 0.2078, 0.2196],
         [0.9922, 0.9922, 0.9647,  ..., 0.1176, 0.0941, 0.1333],
         [0.9961, 0.9922, 1.0000,  ..., 0.1647, 0.1294, 0.1373]],

        [[0.5765, 0.5882, 0.5843,  ..., 0.5490, 0.5569, 0.5608],
         [0.5686, 0.5804, 0.5765,  ..., 0.5490, 0.5529, 0.5529],
         [0.5608, 0.5569, 0.5647,  ..., 0.5569, 0.5490, 0.5529],
         ...,
         [0.7961, 0.8039, 0.7490,  ..., 0.1373, 0.1882, 0.2000],
         [0.9961, 0.9961, 0.9608,  ..., 0.1137, 0.1137, 0.1529],
         [0.9922, 0.9922, 1.0000,  ..., 0.1608, 0.1059, 0.1216]],

        [[0.6235, 0.6353, 0.6314,  ..., 0.5922, 0.6000, 0.6118],
         [0.6078, 0.6235, 0.6196,  ..., 0.5804, 0.5882, 0.6000],
         [0.6039, 0.6118, 0.6196,  ..., 0.5843, 0.584

In [333]:

print(len(txt_ds))
print(txt_ds[0]) # word idx
print(txt_ds.get_vocab_size()) # vocab size

215
tensor([  101,  1037,  2158,  1998,  2450,  2729,  2005,  2019, 10527,  2247,
         1996,  2217,  1997,  1037,  2303,  1997,  2300,  1012,   102,     0,
            0,     0,     0,     0,     0,     0])
30522


In [334]:
import server
import client_cnn
import client_lstm
from torch import nn
from torch import optim
import torch as t

import importlib
importlib.reload(server)
importlib.reload(client_cnn)
importlib.reload(client_lstm)

cnn = client_cnn.CNNModel()
lstm = client_lstm.LSTMModel(vocab_size=txt_ds.get_vocab_size())
head = server.ModelHead()

class Model(nn.Module):
    def __init__(self, cnn, lstm, head):
        super(Model, self).__init__()
        self.cnn = cnn
        self.lstm = lstm
        self.head = head

        self.fc = nn.Sequential(
            nn.Linear(16, 8),
            nn.ReLU(),
        )

    def forward(self, x_cnn, x_lstm):
        x_cnn = self.cnn(x_cnn)
        x_lstm = self.lstm(x_lstm)
        return self.head(self.fc(t.cat([x_cnn, x_lstm], dim=1)))



In [335]:
import torch as t
img_batch = [img_ds[i][0] for i in range(len(img_ds))]
txt_batch = [txt_ds[i] for i in range(len(txt_ds))]
labels = [img_ds[i][1] for i in range(len(img_ds))]


img_tens = t.stack(img_batch).to(device='cuda')
txt_tens = t.stack(txt_batch).to(device='cuda')
labels_tens = t.stack(labels).float().to(device='cuda')
print(img_tens.shape)
print(txt_tens.shape)
print(labels_tens.shape)

torch.Size([215, 3, 224, 224])
torch.Size([215, 26])
torch.Size([215])


In [336]:
import numpy as np
import torch as t
epochs = 100 
batch_size = 64 

model = Model(cnn, lstm, head).to('cuda')

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
shuffle =  t.randperm(img_tens.size()[0])

train_idx = int(0.9 * img_tens.size()[0])
train_img = img_tens[shuffle[:train_idx]]
train_txt = txt_tens[shuffle[:train_idx]]
train_labels = labels_tens[shuffle[:train_idx]]

test_img = img_tens[shuffle[train_idx:]]
test_txt = txt_tens[shuffle[train_idx:]]
test_labels = labels_tens[shuffle[train_idx:]]

print(f"Training on {train_img.size()[0]} samples")
for e in range(epochs):
    
    perm =  t.randperm(train_img.size()[0])
    for i in range(0, train_img.size()[0], batch_size):
        idx = perm[i:i+batch_size]
        batch_img, batch_txt, batch_labels = train_img[idx], train_txt[idx], train_labels[idx]

        pred = model(batch_img, batch_txt)
        loss = loss_fn(pred, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {e}: Loss {loss}")




Training on 193 samples
Epoch 0: Loss 0.8068667650222778
Epoch 1: Loss 0.7496309280395508
Epoch 2: Loss 0.743154764175415
Epoch 3: Loss 0.7360532283782959
Epoch 4: Loss 0.6602200269699097
Epoch 5: Loss 0.7287405133247375
Epoch 6: Loss 0.6672695875167847
Epoch 7: Loss 0.6663392782211304
Epoch 8: Loss 0.7264100909233093
Epoch 9: Loss 0.6628686785697937
Epoch 10: Loss 0.7344264388084412
Epoch 11: Loss 0.7243138551712036
Epoch 12: Loss 0.7099851369857788
Epoch 13: Loss 0.7106583118438721
Epoch 14: Loss 0.6737459897994995
Epoch 15: Loss 0.6932549476623535
Epoch 16: Loss 0.6781238317489624
Epoch 17: Loss 0.6704908013343811
Epoch 18: Loss 0.7005823254585266
Epoch 19: Loss 0.6808908581733704
Epoch 20: Loss 0.6629542708396912
Epoch 21: Loss 0.5714950561523438
Epoch 22: Loss 0.498330295085907
Epoch 23: Loss 0.848120927810669
Epoch 24: Loss 0.5352973341941833
Epoch 25: Loss 0.4718404710292816
Epoch 26: Loss 0.5632936358451843
Epoch 27: Loss 0.4828541874885559
Epoch 28: Loss 1.0157978534698486
Epo

In [337]:
# check model prediction agains train_labels
from torchmetrics.classification import BinaryAccuracy
pred = model(test_img, test_txt)
# pred = pred.detach().numpy()
# pred = t.Tensor(np.where(pred > 0.5, 1, 0))
metrics = BinaryAccuracy().to('cuda')
print(f"Accucary on Test {metrics(pred.detach(), test_labels.detach())}")

Accucary on Test 0.7272727489471436
